# Exercise - Decision Tree

This data set for this exercise is from healthcare. It contains data about 10 years of clinical care at 130 US Hospitals. Each row represents a single patient. The columns include the characteristics of deidentified diabetes patients. This is a binary classification task: predict whether a diabetes patient is readmitted to the hospital within 30 days of their discharge (1=Yes, 0=No). This is an important performance metric for hospitals as they try to minimize these types of readmissions.

## Description of Variables

The description of variables are provided in "Healthcare (small) - Data Dictionary.docx"

## Goal

Use the **healthcare_small.csv** data set and build a model to predict **readmitted**. 

# Read and Prepare the Data

In [6]:
# Common imports

import pandas as pd
import numpy as np

np.random.seed(42)

# Get the data

In [9]:
#We will predict the "price" value in the data set:

diabetes = pd.read_csv("healthcare_small.csv")
diabetes.head()

,race,gender,age,admission_type,discharge_disposition,admission_source,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,readmitted
0,Other,Female,70-80,2,3,1,14,32,3,15,0,0,0,9,1
1,Caucasian,Female,80-90,1,3,5,4,44,0,15,0,0,0,9,0
2,AfricanAmerican,Male,50-60,5,1,1,6,29,1,15,0,0,0,9,1
3,Caucasian,Female,50-60,1,1,6,3,47,0,10,0,0,0,4,0
4,AfricanAmerican,Female,40-50,3,1,1,4,92,0,15,0,0,0,7,0


# Split data (train/test)

In [12]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(diabetes, test_size=0.3)

# Data Prep

In [15]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import GridSearchCV


## Separate the target variable 

In [18]:
train_target = train['readmitted']
test_target = test['readmitted']

train_inputs = train.drop(['readmitted'], axis=1)
test_inputs = test.drop(['readmitted'], axis=1)

## Feature Engineering: Derive a new column

Examples:
- Whether the patient had any emergency visits or not
- Ratio of inpatient visits to outpatient visits
- Ratio of time in hospital to number of medications
- (etc.)

In [21]:
def new_col(df):
    #Create a copy so that we don't overwrite the existing dataframe
    df1 = df.copy()
    
    df1['reviews_per_day'] = (df1['time_in_hospital']/df1['num_medications'])
    
    return df1[['reviews_per_day']]


In [23]:
#Let's test the new function:

# Send the train set to the function we created
new_col(train_inputs)

,reviews_per_day
4166,0.210526
5546,0.500000
2957,0.105263
6329,0.200000
565,0.250000
...,...
5734,0.322581
5191,0.153846
5390,0.608696
860,0.071429


In [25]:
train_target

4166    1
5546    0
2957    1
6329    0
565     0
       ..
5734    1
5191    1
5390    0
860     1
7270    0
Name: readmitted, Length: 6066, dtype: int64

##  Identify the numeric, binary, and categorical columns

In [28]:
train_inputs['admission_source'] = train_inputs['admission_source'].astype('object')
train_inputs['admission_type'] = train_inputs['admission_type'].astype('object')
train_inputs['discharge_disposition'] = train_inputs['discharge_disposition'].astype('object')

test_inputs['admission_source'] = test_inputs['admission_source'].astype('object')
test_inputs['admission_type'] = test_inputs['admission_type'].astype('object')
test_inputs['discharge_disposition'] = test_inputs['discharge_disposition'].astype('object')


In [30]:
# Identify the numerical columns
numeric_columns = train_inputs.select_dtypes(include=[np.number]).columns.to_list()

# Identify the categorical columns
categorical_columns = train_inputs.select_dtypes('object').columns.to_list()

In [32]:
numeric_columns

['time_in_hospital',
 'num_lab_procedures',
 'num_procedures',
 'num_medications',
 'number_outpatient',
 'number_emergency',
 'number_inpatient',
 'number_diagnoses']

In [34]:
categorical_columns

['race',
 'gender',
 'age',
 'admission_type',
 'discharge_disposition',
 'admission_source']

In [36]:
feat_eng_columns = ['time_in_hospital','num_medications']

# Pipeline

In [39]:
numeric_transformer = Pipeline(steps=[
                ('imputer', SimpleImputer(strategy='median')),
                ('scaler', StandardScaler())])

In [41]:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='unknown')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [43]:
# Create a pipeline for the transformed column here
my_new_column = Pipeline(steps=[('my_new_column', FunctionTransformer(new_col)),
                               ('scaler', StandardScaler())])



In [45]:
# First, modify the order of operations in the ColumnTransformer
preprocessor = ColumnTransformer([
        # Put the transformation that creates the new column first
        ('trans', my_new_column, ['time_in_hospital','num_medications']),  # Pass the columns needed for calculation
        ('num', numeric_transformer, numeric_columns),
        ('cat', categorical_transformer, categorical_columns)
    ],   
    remainder='passthrough')

# Transform: fit_transform() for TRAIN

In [48]:
#Fit and transform the train data
train_x = preprocessor.fit_transform(train_inputs)

train_x

<6066x65 sparse matrix of type '<class 'numpy.float64'>'
	with 90990 stored elements in Compressed Sparse Row format>

In [50]:
train_x.shape

(6066, 65)

# Tranform: transform() for TEST

In [53]:
# Transform the test data
test_x = preprocessor.transform(test_inputs)

test_x

<2600x65 sparse matrix of type '<class 'numpy.float64'>'
	with 38997 stored elements in Compressed Sparse Row format>

In [55]:
test_x.shape

(2600, 65)

# Calculate the Baseline

In [58]:
from sklearn.dummy import DummyClassifier

dummy_clf = DummyClassifier(strategy="most_frequent")

dummy_clf.fit(train_x, train_target)

DummyClassifier(strategy='most_frequent')

In [60]:
from sklearn.metrics import accuracy_score

In [62]:
#Baseline Train Accuracy
dummy_train_pred = dummy_clf.predict(train_x)

baseline_train_acc = accuracy_score(train_target, dummy_train_pred)

print('Baseline Train Accuracy: {}' .format(baseline_train_acc))

Baseline Train Accuracy: 0.5313221233102539


In [64]:
#Baseline Test Accuracy
dummy_test_pred = dummy_clf.predict(test_x)

baseline_test_acc = accuracy_score(test_target, dummy_test_pred)

print('Baseline Test Accuracy: {}' .format(baseline_test_acc))

Baseline Test Accuracy: 0.5365384615384615


# Train a DT model

In [67]:
from sklearn.tree import DecisionTreeClassifier 

tree_clf = DecisionTreeClassifier(max_depth=5)

tree_clf.fit(train_x, train_target)

DecisionTreeClassifier(max_depth=5)

### Calculate the accuracy

In [70]:
from sklearn.metrics import accuracy_score
#Train accuracy:
train_y_pred = tree_clf.predict(train_x)

print(accuracy_score(train_target, train_y_pred))

0.6264424662050775


In [72]:
#Test accuracy:
test_y_pred = tree_clf.predict(test_x)

print(accuracy_score(test_target, test_y_pred))

0.605


# Train another DT model (with different parameters)

In [75]:
from sklearn.tree import DecisionTreeClassifier 

tree_clf2 = DecisionTreeClassifier(max_depth=6, min_samples_leaf=4)

tree_clf2.fit(train_x, train_target)

DecisionTreeClassifier(max_depth=6, min_samples_leaf=4)

In [77]:
#Train accuracy:
train_y_pred = tree_clf2.predict(train_x)

print(accuracy_score(train_target, train_y_pred))

0.6335311572700296


In [79]:
#Test accuracy:
test_y_pred = tree_clf2.predict(test_x)

print(accuracy_score(test_target, test_y_pred))

0.6103846153846154


In [443]:
from sklearn.metrics import accuracy_score, classification_report
# Another Decision Tree model with different parameters
tree_clf2 = DecisionTreeClassifier(
    max_depth=8,
    min_samples_split=10,
    min_samples_leaf=4,
    random_state=42
)
tree_clf2

DecisionTreeClassifier(max_depth=8, min_samples_leaf=4, min_samples_split=10,
                       random_state=42)

In [445]:
# Train the model
tree_clf2.fit(train_x, train_target)

# Predict on training and testing data
train_pred2 = tree_clf2.predict(train_x)
test_pred2 = tree_clf2.predict(test_x)

### Calculate the accuracy

In [448]:

# Evaluate the model
train_accuracy2 = accuracy_score(train_target, train_pred2)
test_accuracy2 = accuracy_score(test_target, test_pred2)

print('Second Model Results (Classifier):')
print('Train Accuracy: {:.2f}%'.format(train_accuracy2 * 100))
print('Test Accuracy: {:.2f}%'.format(test_accuracy2 * 100))

# Optional: Detailed classification report
print('\nClassification Report (Test Set):')
print(classification_report(test_target, test_pred2))

Second Model Results (Classifier):
Train Accuracy: 65.22%
Test Accuracy: 60.96%

Classification Report (Test Set):
              precision    recall  f1-score   support

           0       0.62      0.72      0.66      1395
           1       0.60      0.49      0.54      1205

    accuracy                           0.61      2600
   macro avg       0.61      0.60      0.60      2600
weighted avg       0.61      0.61      0.60      2600



# Optional: try grid search